In [2]:
import numpy as np
import pandas as pd

import re
from tqdm import tqdm

import collections

from sklearn.cluster import KMeans

from nltk.stem import WordNetLemmatizer  # For Lemmetization of words
from nltk.corpus import stopwords  # Load list of stopwords
from nltk import word_tokenize # Convert paragraph in tokens

import pickle
import sys


from gensim import models 

In [3]:
text_data = pd.read_csv("Precily_Text_Similarity.csv")
print("Shape of text_data : ", text_data.shape)
text_data.head()

Shape of text_data :  (3000, 2)


,text1,text2
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...
3,hearts of oak 3-2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...


In [4]:
text_data.isnull().sum()

text1    0
text2    0
dtype: int64

In [5]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [6]:
preprocessed_text1 = []

# tqdm is for printing the status bar

for sentance in tqdm(text_data['text1'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)

    sent = ' '.join(e for e in sent.split() if e not in stopwords.words('english'))
    preprocessed_text1.append(sent.lower().strip())

100%|██████████████████████████████████████████████████████████████████████████████| 3000/3000 [05:47<00:00,  8.64it/s]


In [7]:
text_data['text1'] = preprocessed_text1
text_data.head(3)

,text1,text2
0,broadband challenges tv viewing number europea...,gardener wins double in glasgow britain s jaso...
1,rap boss arrested drug find rap mogul marion s...,amnesty chief laments war failure the lack of ...
2,player burn worries robinson england coach and...,hanks greeted at wintry premiere hollywood sta...


In [8]:
from tqdm import tqdm
preprocessed_text2 = []

# tqdm is for printing the status bar
for sentance in tqdm(text_data['text2'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
   
    sent = ' '.join(e for e in sent.split() if e not in stopwords.words('english'))
    preprocessed_text2.append(sent.lower().strip())

100%|██████████████████████████████████████████████████████████████████████████████| 3000/3000 [05:49<00:00,  8.59it/s]


In [9]:
text_data['text2'] = preprocessed_text2

text_data.head(3)

,text1,text2
0,broadband challenges tv viewing number europea...,gardener wins double glasgow britain jason gar...
1,rap boss arrested drug find rap mogul marion s...,amnesty chief laments war failure lack public ...
2,player burn worries robinson england coach and...,hanks greeted wintry premiere hollywood star t...


In [10]:
def word_tokenizer(text):
            #tokenizes and stems the text
            tokens = word_tokenize(text)
            lemmatizer = WordNetLemmatizer() 
            tokens = [lemmatizer.lemmatize(t) for t in tokens]
            return tokens

In [11]:
url="GoogleNews-vectors-negative300.bin.gz"
wordmodel= models.KeyedVectors.load_word2vec_format(url, binary=True)

In [12]:
similarity = [] # List for store similarity score



for ind in text_data.index:
    
        s1 = text_data['text1'][ind]
        s2 = text_data['text2'][ind]
        
        if s1==s2:
                 similarity.append(0.0) # 0 means highly similar
                
        else:   

            s1words = word_tokenizer(s1)
            s2words = word_tokenizer(s2)
            
           
            
            vocab = wordmodel #the vocabulary considered in the word embeddings
            
            if len(s1words and s2words)==0:
                    similarity.append(1.0)

            else:
                
                for word in s1words.copy(): #remove sentence words not found in the vocab
                    if (word not in vocab):
                           
                            
                            s1words.remove(word)
                        
                    
                for word in s2words.copy(): #idem

                    if (word not in vocab):
                           
                            s2words.remove(word)
                            
                            
                similarity.append((1-wordmodel.n_similarity(s1words, s2words)))

In [13]:
final_score = pd.DataFrame({'Similarity_score':similarity})
final_score.head()

,Similarity_score
0,0.314564
1,0.366671
2,0.282510
3,0.383152
4,0.207984


In [14]:
final_score.to_csv('final_score.csv',index=True)